In [ ]:
!pip install pypdf
!pip install faiss-cpu
!pip install langchain
!pip install langgraph
!pip install langchain-core
!pip install langchain-community
!pip install langchain-openai




In [ ]:
#Import des clés api OpenAI et Tavily
from google.colab import userdata
import os
openai_api_key = userdata.get('openai_api_key')
tavily_api_key = userdata.get('tavily_api_key')


if openai_api_key :
  os.environ['OPENAI_API_KEY'] = openai_api_key
  print("Cle openAi disponible")

else :
  print("Cle openAi non disponible veuillez la configurer dans votre environnement virtuel ou dans colab secrets")
  os.environ['OPENAI_API_KEY'] =""

if tavily_api_key :
  os.environ['TAVILY_API_KEY'] = tavily_api_key
  print("Cle Tavily disponible")
else :
  print("Cle Tavily non disponible veuillez la configurer dans votre environnement virtuel ou dans colab secrets")
  os.environ['TAVILY_API_KEY'] =""


In [ ]:
#Import pour le  chargement des données le decoupage et les embeddings

#Diviser les grands documents en chunks gérables

from langchain.text_splitter import RecursiveCharacterTextSplitter


#Charger les documents directement depuis des pasges web
from langchain_community.document_loaders import WebBaseLoader

#Modèle d'embeddings OpenAI pour la conversion texte --> vecteur

from langchain.embeddings.openai import OpenAIEmbeddings

In [ ]:
#Construire l'index à partir de PDFs
from langchain_community.document_loaders import PyPDFLoader


# Etape 1: definir les embeddings
embeddings = OpenAIEmbeddings()


# Etape 2: charger les documents pdf au lieu D'URl
fichiers_pdf=[

]
doc = []
for fichier in fichiers_pdf:
  loader = PyPDFLoader(fichier)
  doc.extend(loader.load()) #pypdf retourne deja une liste de documents


print(f'{len(doc)} documents chargés depuis {len(fichiers_pdf)} fichiers pdf')


# Etape 3: Decoupage en chunk
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size =1200 #12000 caractères maximum par chunk
    chunk_overlap = 100 #pour eviter de couper des phrases importantes,on garde un chevauchement de 100 carateres entre les chunks
)

docs_split = text_splitter.split_documents(doc)
print(f'{len(docs_split)} chunks créés')



#Etape 4: Construction de la base de données vectorielle avec (FAISS)

vectorstore = FAISS.from_documents(
    documents=docs_split
    embedding=embeddings
    )


#Etape 5: Création du retriever
retriever = vectorstore.as_retriever()

print("Retriever prêt")

In [ ]:
#Evaluateur de Recuperation (Retrieval grader)

from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import chatOpenAI
from pydantic import BaseModel, Field



# Etape 1: definir le schema de sortie
class GradeDocuments(BaseModel):
  """score binaire pour verifier la pertinence des documents recuperer"""
  binary_score: str = Field(description="Les documents sont pertinents pour la question ? : 'yes' ou 'no'")


#Etape 2: Initialiser le LLM avec sortie structuree
'''On initialise le LLM avec une sortie structurée et la température à 0 pour garantir que la réponse
du LLM respecte à chaque fois la structure definitie initialement grace à pydantic '''


llm= chatOpenAI(
    model ="gpt-4.1-nano-2025-04-14",
    temperature=0
)
structured_llm_grader =llm.with_structured_output(GradeDocuments)

# Etape 3: definir le prompt d'évaluation
 system_msg="""
 Tu es un évaluateur qui évalue la pertinence d'un document récupéré par rapport à une question posée par un utilisateur.
 Si le document contient des mots clés ou une signification sémantique liés à la question, note le comme pertinent.
 Donne uniquement un score binaire: 'yes' or 'no'
 """

 grade_prompt = ChatPromptTemplate.from_messages([
     ("system", system_msg),
     ("human", "Document récupéré :\n\n{document}\n\nQuestion utilisateur :\n\n{question}")
 ])



# Etape 4: Construire la chaine d'évaluation
retrieval_grader= grade_prompt | structured_llm_grader


#Etape 5: tester l'évaluateur
question="QU'est ce que le RAG"
docs=retriever.invoke(question)


# On selectionne un chunk récupéré pour l'évaluation
doc_txt= doc[1].page_content


# exécuter l'évaluateur
result=retrieval_grader.invoke({"document": doc_txt, "question": question})
print("Resultat de l'évaluation:",result)









In [ ]:
#Generer la réponse finale

from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain _openai import chatOpenAI


#Étape 1: charger le template de Prompt RAG


#rlm/rag-prompt est un prompt communautaire conçu pour les réponses de type RAG
prompt=hub.pull("rlm/rag-prompt")

ETape 2: Initialiser le LLM

llm= chatOpenAI(
    model ="gpt-4.1-nano-2025-04-14",
    temperature=0.7   #pour que le LLM soit plus creatif
)

#Etape 3: Definir l'aide au formatage des documents

format_docs(docs):
  """Il s'agit de joindre plusieurs documents en une seule chaine pour la variable de contexte."""
  return "\n\n".join(doc.page_content for doc in docs)


#Etape 4: Construire la chaine RAG

rag_chain=prompt | llm | StrOutputParser()


#Etape 5:Exécuter la génération en utilisant  le contenu des documents récupérés

context_text=format_docs(docs)


generation=rag_chain.invoke({
    "context": context_text,
    "question": question
})

print(generation)




